### To use Invoke Ai, launch the code in every form below.
You will need an account on [ngrok](https://dashboard.ngrok.com/login)

Requires 13 Gb of space on a Google disk

####**[Original Repo](https://github.com/peaashmeter/invoke-ai-gui-colab)**
####**[This Repo](https://github.com/Lycantant/invoke-ai-gui-colab/)**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title During this stage, Colab crashes, it's normal
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title Installing Invoke Ai { display-mode: "form" }
%cd /home

!git clone -n https://github.com/invoke-ai/InvokeAI.git
%cd InvokeAI
!git checkout tags/v2.2.5

!ln -sf environments-and-requirements/environment-lin-cuda.yml environment.yml
!ls -la

%cd ..

!git clone https://github.com/Lycantant/invoke-ai-gui-colab.git

%cd invoke-ai-gui-colab

%cp cross_attention_control.py ../InvokeAI/ldm/models/diffusion/cross_attention_control.py
%cp globals.py ../InvokeAI/ldm/invoke/globals.py

In [ ]:
#@title Installation normally takes ~10 minutes { display-mode: "form" }
%cd ../InvokeAI
!pip install pyngrok --quiet
!conda env update
!source activate invokeai ; python scripts/configure_invokeai.py --yes

%cp ../invoke-ai-gui-colab/models.yaml /root/invokeai/configs

In [ ]:
#@title Installation of Stable_Diffusion_15_default (requires 13 Gb of space on a Google disk){ display-mode: "form" }
import os

os.system('cd /')
os.system('mkdir -p /root/invokeai/models/ldm/stable-diffusion-v1')

# sd 1.4
if os.path.exists('/content/drive/MyDrive/models/sd-v1-4.ckpt'):
    print('Model sd-v1-4.ckpt already exists')
else:
    print('Downloading Stable-Diffusion 1.4')
    os.system('mkdir -p /content/drive/MyDrive/models/')
    os.system('wget -O /content/drive/MyDrive/models/sd-v1-4.ckpt https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt')

# sd 1.5
if os.path.exists('/content/drive/MyDrive/models/v1-5-pruned-emaonly.ckpt'):
    print('Model v1-5-pruned-emaonly.ckpt already exists')
else:
    print('Downloading Stable-Diffusion 1.5')
    os.system('mkdir -p /content/drive/MyDrive/models/')
    os.system('wget -O /content/drive/MyDrive/models/v1-5-pruned-emaonly.ckpt https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt')

# seekArtMega_v1
if os.path.exists('/content/drive/MyDrive/models/seekArtMega_v1-ckpt.ckpt'):
  print('Model seekArtMega_v1-ckpt.ckpt already exists')
else:
  print('Downloading seekArtMega_v1-ckpt.ckpt')
  os.system('wget -O /content/drive/MyDrive/models/seekArtMega_v1-ckpt.ckpt https://huggingface.co/coreco/seek.art_MEGA/resolve/main/seek_art_mega_v1.ckpt')

# openjourney v4
if os.path.exists('/content/drive/MyDrive/models/mdjrny-v4.ckpt'):
  print('Model mdjrny-v4.ckpt already exists')
else:
  print('Downloading mdjrny-v4.ckpt')
  os.system('wget -O /content/drive/MyDrive/models/mdjrny-v4.ckpt https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt')

# vae
if os.path.exists('/content/drive/MyDrive/models/vae-ft-mse-840000-ema-pruned.ckpt'):
  print('Model vae-ft-mse-840000-ema-pruned.ckpt already exists')
else:
  print('Downloading SD-VAE')
  os.system('wget -O /content/drive/MyDrive/models/vae-ft-mse-840000-ema-pruned.ckpt https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt')

# if not not os.path.exists('/content/drive/MyDrive/models/vae-ft-mse-840000-ema-pruned.ckpt'):
# heelllp, what does `if not not` mean? I'm not familiar with programming, especially python -_-
if not os.path.exists('/content/drive/MyDrive/models/vae-ft-mse-840000-ema-pruned.ckpt'):
    print('vae-ft-mse-840000-ema-pruned.ckpt doesn\'t exist')
else:
    print('Make sure you accept the terms at https://huggingface.co/stabilityai/sd-vae-ft-mse-original')

# sd 1.4
%mkdir -p /root/invokeai/models/ldm/stable-diffusion-v1/
!ln -s  /content/drive/MyDrive/models/sd-v1-4.ckpt /root/invokeai/models/ldm/stable-diffusion-v1/sd-v1-4.ckpt
!ls -l /root/invokeai/models/ldm/stable-diffusion-v1/sd-v1-4.ckpt

# sd 1.5
!ln -s  /content/drive/MyDrive/models/v1-5-pruned-emaonly.ckpt /root/invokeai/models/ldm/stable-diffusion-v1/v1-5-pruned-emaonly.ckpt
!ls -l /root/invokeai/models/ldm/stable-diffusion-v1/v1-5-pruned-emaonly.ckpt

# seekArtMega v1
!ln -s  /content/drive/MyDrive/models/seekArtMega_v1-ckpt.ckpt /root/invokeai/models/ldm/stable-diffusion-v1/seekArtMega_v1-ckpt.ckpt
!ls -l /root/invokeai/models/ldm/stable-diffusion-v1/seekArtMega_v1-ckpt.ckpt

# openjourney v4
!ln -s  /content/drive/MyDrive/models/mdjrny-v4.ckpt /root/invokeai/models/ldm/stable-diffusion-v1/mdjrny-v4.ckpt
!ls -l /root/invokeai/models/ldm/stable-diffusion-v1/mdjrny-v4.ckpt

# vae
!ln -s  /content/drive/MyDrive/models/vae-ft-mse-840000-ema-pruned.ckpt /root/invokeai/models/ldm/stable-diffusion-v1/vae-ft-mse-840000-ema-pruned.ckpt
!ls -l /root/invokeai/models/ldm/stable-diffusion-v1/vae-ft-mse-840000-ema-pruned.ckpt

In [ ]:
#@title Running Invoke AI service { display-mode: "both" }
#@markdown Go to [ngrok](https://dashboard.ngrok.com/get-started/your-authtoken) , and paste your authtoken here.

ngrok_token = "" #@param {type:"string"}
nsfw_checker = 0 #@param {type:"slider", min:0, max:1, step:1}
used_vae = "vae-ft-mse-840000-ema-pruned" #@param ["vae-ft-mse-840000-ema-pruned"]

import os
from pyngrok import ngrok

ngrok.kill()
ngrok.set_auth_token(ngrok_token)
public_url = ngrok.connect(9090).public_url
print(f'Invoke Ai public url: {public_url}')

%cd /home/InvokeAI

model_name = "Stable_Diffusion_15_default"
if used_vae == "vae-ft-mse-840000-ema-pruned":
    model_name = "Stable_Diffusion_15_default"

if nsfw_checker:
    !source activate invokeai ; python scripts/invoke.py --web --model $model_name
else:
    !source activate invokeai ; python scripts/invoke.py --web --no-nsfw_checker --model $model_name

### Wait until the local url (`http://localhost:9090`) is available.
### Then click/open public url, it will look something like this `http://3877-35-204-151-175.ngrok.io`
### Refresh the page if you see a blank white screen
---